In [168]:
import requests
import unittest
import re
from bs4 import BeautifulSoup
import pandas as pd

def getAttributesFromPage(company, url):
    result = requests.get(url)
    result.status_code
    print( "Getting data from "+company)
    if result.status_code == 200:
        soup= BeautifulSoup(result.content, "html.parser")
        trs=soup.find("td" , text="Quarter Ending Dec-18")
        saleDecember = trs.findNext("td").findNext("td").text
  
        divprice= soup.findAll("div", attrs={"class":"sectionQuoteDetail"})[1]
        spanprice= divprice.find("span", attrs={"class":u"valueContent"})
        currentprice= spanprice.find("span").text.strip()
        changerate= spanprice.find("span", "valueContentPercent").find("span").text.strip()
 
        shares=soup.find("strong" , string="% Shares Owned:").parent.parent.findAll("td")[1].text
        dividend=soup.find("td" , string="Dividend Yield").findNext("td").parent.findAll("td")
        global df2
        df2.loc[-1] = [company,currentprice, changerate, saleDecember, shares, dividend[1].text, dividend[2].text, 
                               dividend[3].text]  # adding a row
        df2.index = df2.index + 1  # shifting index
        df2 = df2.sort_index()  # sorting by index


website_prefix = "https://www.reuters.com/finance/stocks/financial-highlights/"
companies={"LVMH":"LVMH.PA",
          "AIRBUS":"AIR.PA",
          "DANONE":"DANO.PA"}
columns=['Company', 'CurrentPrice', 'ChangeRate', 'SaleDecember', 'ShareOwned', 'Dividend Company', 
                            'Dividend industry','Dividend sector']
df2 = pd.DataFrame(columns=columns)
for key,value in companies.items():
    getAttributesFromPage(key, website_prefix+value)
df2.head()

Getting data from LVMH
Getting data from AIRBUS
Getting data from DANONE


,Company,CurrentPrice,ChangeRate,SaleDecember,ShareOwned,Dividend Company,Dividend industry,Dividend sector
0,DANONE,€-1.40,(-2.14%),"6,072.60",50.60%,2.90,2.78,2.48
1,AIRBUS,€-1.66,(-1.68%),"23,493.00",43.53%,1.45,1.34,1.64
2,LVMH,€-3.35,(-1.26%),"13,667.70",20.63%,1.82,1.67,2.15


 Share owned are: 50.61%


Dividend Yield for company : 1.92
Dividend Yield for industry : 1.70
Dividend Yield for sector : 2.60
